In [1]:
import sys
import string
import codecs
import mmap
import re
import random
import logging
from importlib import reload
import os
import gensim
from gensim.models.phrases import Phrases, Phraser
import smart_open
import statistics
import numpy as np
from sklearn.model_selection import train_test_split
import collections
import nltk
import sent2vec
import sklearn
import itertools
cwd = os.path.dirname(os.getcwd())
print(cwd)

/home/joy/Thesis


# Preprocess training data

Each line is preprocessed for training the Sent2Vec model. Sent2Vec trains on untokenised sentence documents.

In [2]:
def preprocess_data(fname_source, fname_des):
    source = smart_open.open(fname_source, encoding="iso-8859-1")
    des = smart_open.open(fname_des, 'w')
    for line in source.readlines():
        line = line.strip()
        line = str(line.lower())
        line = nltk.word_tokenize(line)
        line = [word for word in line if (word.isalpha() or word == '.' or word == ',')]
        for item in line:
        #print(item)
            des.write("%s " % item)
        des.write("\n")
    return open(fname_des).readlines()

#preprocess_data(cwd + '/Data/quinev05_input_word.txt', 'training_data_word2.txt')
#preprocess_data(cwd + '/Data/quinev05_input_lemma.txt', 'training_data_lemma.txt')   

# Train Sent2Vec model on training data

Trains Sent2Vec models for each gridsearch combination.

In [3]:
# #!/bin/bash

# griddict = {
#         'e': [100, 200, 300], #epochs
#         'd': [64, 128, 256], #vec dim
#         'lr': [0.15, 0.2, 0.25, 0.3], #learning rate
#         'minCount': [0, 5],
#         'loss' : ['ns'],
#         'drop': [2],
#         'sub' : [1.0],
#      }

# keys, values = zip(*griddict.items())
# calls = []
# i = 0
# total = str(len(list(itertools.product(*values))))
# print("number of combinations: " + total)
# for v in itertools.product(*values):
#     experiment = dict(zip(keys, v))
#     # create filname from griddict values
#     outfilename = '/media/joy/Seagate_usb/Models/ns-word/s2vquine05_tune_d' + str(experiment['d']) + '_e' + str(experiment['e']) + '_lr' + str(experiment['lr']) + '_min' + str(experiment['minCount']) + '_loss_' + str(experiment['loss']) + '_drop_' + str(experiment['drop']) + '_sub' + str(experiment['sub'])
#     modelfilename = outfilename #+ '.bin'

#     # CHANGE TO CORRECT TRAINING DATA FILE!
#     command ="! ./fasttext sent2vec -input training_data_word.txt -output " + modelfilename + " -dim " + str(experiment['d']) + " -epoch " + str(experiment['e']) + " -lr " + str(experiment['lr']) + " -minCount " + str(experiment['minCount']) + " -loss " + str(experiment['loss']) + " -dropoutK " + str(experiment['drop']) + " -t " + str(experiment['sub'])
#     print(i, "/", total, command)
#     os.system(command)
#     i += 1

# Evaluate model on queries

In [10]:
def tests2vqueryrun(train_file, test_annot, modelfile, query, n, docid):
#This function can run the same evaluation multiple times, which is necessary for inferred query vectors, as they can be quite different each time. When using additive or trained query vectors, it just runs the evaluation once and the aggregate statistics don't mean anything.

    
    #loading the model
    model = sent2vec.Sent2vecModel()
    model.load_model(modelfile)
    
    # CHANGE PATH HERE
    no_path = modelfile.replace('/media/joy/Seagate_usb/Models/Sent2Vec/best-word', '')
    # Send log output to txt file
    # CHANGE PATH HERE
    txtfile = '/media/joy/Seagate_usb/Logs/Sent2Vec/ns/Test/word/paragraphs/9097' + no_path.replace(".bin", ".txt")
    logging.shutdown()
    reload(logging)
    logging.basicConfig(filename=txtfile,
                    format='%(asctime)s %(message)s',
                    filemode='w')
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    
    # Preprocess training data file.
    test_corpus = preprocess_data(cwd + '/Data/' + train_file, 'training_data.txt')

    a = codecs.open(test_annot, encoding="utf-8")
    annot_corpus = a.readlines()
    annot_corpus = [i.split() for i in annot_corpus]

    apscore10_eval = []
    apscore100_eval = []
    apscore500_eval = []
    f10_eval = []
    f100_eval = []
    f500_eval = []
    pc10_eval = []
    pc100_eval = []
    pc500_eval = []
    pcr_eval = []
    pcr_r_eval = []
    pcr_m_eval = []
    pcr_i_eval = []
    avgrank_all_eval = []
    avgrank_500_eval = []
    avgdist_all_eval = []
    avgdist_500_eval = []

    for i in range(n):
        apscore10, apscore100, apscore500, pc10, pc100, pc500, f10, f100, f500, pcr, pcr_r, pcr_m, pcr_i, avgrank_all, avgrank_500, avgdist_all, avgdist_500 = tests2vquery(test_corpus, annot_corpus, model, query, logger, docid)
        #print(apscore10, apscore100, apscore500, pc10, pc100, pc500, f10, f100, f500, pcr, pcr_r, pcr_m, pcr_i, avgrank_all, avgrank_500, avgdist_all, avgdist_500)
        apscore10_eval.append(apscore10)
        apscore100_eval.append(apscore100)
        apscore500_eval.append(apscore500)
        f10_eval.append(f10)
        f100_eval.append(f100)
        f500_eval.append(f500)
        pc10_eval.append(pc10)
        pc100_eval.append(pc100)
        pc500_eval.append(pc500)
        pcr_eval.append(pcr)
        pcr_r_eval.append(pcr_r)
        pcr_m_eval.append(pcr_m)
        pcr_i_eval.append(pcr_i)
        avgrank_all_eval.append(avgrank_all)
        avgrank_500_eval.append(avgrank_500)
        avgdist_all_eval.append(avgdist_all)
        avgdist_500_eval.append(avgdist_500)

    print(pcr)
    logger.info('Over {} query runs:'.format(n))
    logger.info('Mean Precision @ 10: {}, Mean Precision @ 100: {}, Mean Precision @ 500: {}'.format(sum(pc10_eval)/float(len(pc10_eval)), sum(pc100_eval)/float(len(pc100_eval)), sum(pc500_eval)/float(len(pc500_eval))))
    logger.info('Mean F-score @ 10: {}, Mean F-score @ 100: {}, Mean F-score @ 500: {}'.format(sum(f10_eval)/float(len(f10_eval)), sum(f100_eval)/float(len(f100_eval)), sum(f500_eval)/float(len(f500_eval))))
    logger.info('Mean Average Precision @ 10: {}, Mean Average Precision @ 100: {}, Mean Average Precision @ 500: {}'.format(sum(apscore10_eval)/float(len(apscore10_eval)), sum(apscore100_eval)/float(len(apscore100_eval)), sum(apscore500_eval)/float(len(apscore500_eval))))
    logger.info('Mean Precision @ R: {}'.format(sum(pcr_eval)/float(len(pcr_eval))))
    logger.info('Mean Precision @ R for Relevant results: {}'.format(sum(pcr_r_eval)/float(len(pcr_r_eval))))
    logger.info('Mean Precision @ R for Mildly Relevant results: {}'.format(sum(pcr_m_eval)/float(len(pcr_m_eval))))
    logger.info('Mean Precision @ R for Irrelevant results: {}'.format(sum(pcr_i_eval)/float(len(pcr_i_eval))))
    logger.info('Average rank of targets: {}'.format(sum(avgrank_all_eval)/float(len(avgrank_all_eval))))
    logger.info('Average rank of top 500 targets: {}'.format(sum(avgrank_500_eval)/float(len(avgrank_500_eval))))
    logger.info('Average distance of target to query: {}'.format(sum(avgdist_all_eval)/float(len(avgdist_all_eval))))
    logger.info('Average distance of top 500 targets to query: {}'.format(sum(avgdist_500_eval)/float(len(avgdist_500_eval))))


In [11]:
def most_similar(emb_matrix, qvector, n):
    top_n = []
    matrix = sklearn.metrics.pairwise.cosine_similarity(emb_matrix, qvector, dense_output=True)
    indices = np.argpartition(matrix, -n, axis = 0)[-n:].flatten()
    scores = np.partition(matrix, -n, axis = 0)[-n:].flatten()
    for index, score in zip(indices, scores):
        top_n.append((index, score))
    return sorted(top_n, reverse = True,  key=lambda x: x[1])

def tests2vquery(test_corpus, annot_corpus, model, query, logger, docid):
    #Performs a single query and computes evaluation metrics over it. Supports summed query vectors (add), inferred query vectors (inf) and taking a trained document vector as a query vector (trn).
    #If a docID is provided, it will use that as a query, otherwise it will use 'query'.
    
    annot_hits = []
    annot_hits_m = []
    annot_hits_r = []
    annot_hits_i = []
    for i, line in enumerate(annot_corpus):
        if line:
            if '@' in line[0]:
                if '@1' in line[0]:
                    annot_hits.append(i)
                    annot_hits_r.append(i)
                if '@0' in line[0]:
                    annot_hits.append(i)
                    annot_hits_m.append(i)
                if '@-1' in line[0]:
                    annot_hits_i.append(i)
    #print('annot_hits', annot_hits)
    if docid:
        qvector = model.embed_sentence(test_corpus[docid])

    else:
        #qtokens = gensim.utils.simple_preprocess(query) 
        qvector = model.embed_sentence(query)

    #logger.info('10 most similar words to query vector = {}'.format(model.similar_by_vector(qvector, topn=10)))

    emb_matrix = model.embed_sentences(test_corpus)
    sims = most_similar(emb_matrix, qvector, len(test_corpus))
    sims10 = sims[:10]
    sims100 = sims[:100]
    
    matches = {}
    matches_r = {}
    matches_m = {}
    matches_i = {}
    total = 0
    top10 = 0
    top1 = 0
    top100 = 0
    top500 = 0
    top_r = 0 #For computing precision @ R (recall, the number of target documents)
    eval_avgrank = []
    eval_avg_distance_between = []
    eval_avgrank_all = []
    eval_avg_distance_between_all = []

    total_r = 0
    top10_r = 0
    top1_r = 0
    top100_r = 0
    top500_r = 0
    top_r_r = 0
    eval_avgrank_r = []
    eval_avg_distance_between_r = []
    eval_avgrank_all_r = []
    eval_avg_distance_between_all_r = []

    total_m = 0
    top10_m = 0
    top1_m = 0
    top100_m = 0
    top500_m = 0
    top_r_m = 0
    eval_avgrank_m = []
    eval_avg_distance_between_m = []
    eval_avgrank_all_m = []
    eval_avg_distance_between_all_m = []

    total_i = 0
    top10_i = 0
    top1_i = 0
    top100_i = 0
    top500_i = 0
    top_r_i = 0
    eval_avgrank_i = []
    eval_avg_distance_between_i = []
    eval_avgrank_all_i = []
    eval_avg_distance_between_all_i = []

    total_match = len(annot_hits)
    total_match_r = len(annot_hits_r)
    total_match_m = len(annot_hits_m)
    total_match_i = len(annot_hits_i)
    total = len(annot_corpus)

    #for line in annot_hits:
    #    annotation = line.split(',')
    rank = 1
    for match in sims:
        if match[0] in annot_hits:
            #Count total hits of relevant passages
            matches[match[0]] = (rank,match[1])
            eval_avgrank_all.append(rank)
            eval_avg_distance_between_all.append(match[1])
            if rank < 501:
                eval_avgrank.append(rank)
                eval_avg_distance_between.append(match[1])
                top500 = top500 + 1
                if rank < 101:
                    top100 = top100 + 1
                    if rank < 11:
                        top10 = top10 + 1
                        if rank == 1:
                            top1 = top1 + 1
            if rank < total_match:
                top_r = top_r + 1

        if match[0] in annot_hits_r:
            matches_r[match[0]] = (rank,match[1])
            eval_avgrank_all_r.append(rank)
            eval_avg_distance_between_all_r.append(match[1])
            if rank < 501:
                eval_avgrank_r.append(rank)
                eval_avg_distance_between_r.append(match[1])
                top500_r = top500_r + 1
                if rank < 101:
                    top100_r = top100_r + 1
                    if rank < 11:
                        top10_r = top10_r + 1
                        if rank == 1:
                            top1_r = top1_r + 1
            if rank < total_match_r:
                top_r_r = top_r_r + 1

        if match[0] in annot_hits_m:
            matches_m[match[0]] = (rank,match[1])
            eval_avgrank_all_m.append(rank)
            eval_avg_distance_between_all_m.append(match[1])
            if rank < 501:
                eval_avgrank_m.append(rank)
                eval_avg_distance_between_m.append(match[1])
                top500_m = top500_m + 1
                if rank < 101:
                    top100_m = top100_m + 1
                    if rank < 11:
                        top10_m = top10_m + 1
                        if rank == 1:
                            top1_m = top1_m + 1
            if rank < total_match_m:
                top_r_m = top_r_m + 1
        #rank = rank + 1

        if match[0] in annot_hits_i:
            matches_i[match[0]] = (rank,match[1])
            eval_avgrank_all_i.append(rank)
            eval_avg_distance_between_all_i.append(match[1])
            if rank < 501:
                eval_avgrank_i.append(rank)
                eval_avg_distance_between_i.append(match[1])
                top500_i = top500_i + 1
                if rank < 101:
                    top100_i = top100_i + 1
                    if rank < 11:
                        top10_i = top10_i + 1
                        if rank == 1:
                            top1_i = top1_i + 1
            if rank < total_match_i:
                top_r_i = top_r_i + 1
        rank = rank + 1
        
    i = 0
  
    #top_r = top_r_r + top_r_m
    print(top_r,top_r_r , top_r_m, top_r_i)
    #Compute Average Precision metric for the total, scientistic and modest hits
    ap = []
    ap_r = []
    ap_m = []
    ap_i = []
    for rank in sorted(eval_avgrank):
        i=i+1
        ap.append(i/rank)
        if rank in eval_avgrank_r:
            ap_r.append(i/rank)
        if rank in eval_avgrank_m:
            ap_m.append(i/rank)
        if rank in eval_avgrank_i:
            ap_i.append(i/rank)

    if ap:
        apscore = sum(ap)/float(len(ap))
    else:
        apscore = 0
    if ap_r:
        apscore_r = sum(ap_r)/float(len(ap_r))
    else:
        apscore_r = 0
    if ap_m:
        apscore_m = sum(ap_m)/float(len(ap_m))
    else:
        apscore_m = 0
    if ap_i:
        apscore_i = sum(ap_i)/float(len(ap_i))
    else:
        apscore_i = 0

    i = 0
    ap100 = []
    ap100_r = []
    ap100_m = []
    ap100_i = []
    for rank in sorted(eval_avgrank):
        if rank < 101:
            i=i+1
            ap100.append(i/rank)
            if rank in eval_avgrank_r:
                ap100_r.append(i/rank)
            if rank in eval_avgrank_m:
                ap100_m.append(i/rank)
            if rank in eval_avgrank_i:
                ap100_i.append(i/rank)
    if ap100:
        apscore100 = sum(ap100)/float(len(ap100))
    else:
        apscore100 = 0
    if ap100_r:
        apscore100_r = sum(ap100_r)/float(len(ap100_r))
    else:
        apscore100_r = 0
    if ap100_m:
        apscore100_m = sum(ap100_m)/float(len(ap100_m))
    else:
        apscore100_m = 0
    if ap100_i:
        apscore100_i = sum(ap100_i)/float(len(ap100_i))
    else:
        apscore100_i = 0


    i = 0
    ap10 = []
    ap10_r = []
    ap10_m = []
    ap10_i = []
    for rank in sorted(eval_avgrank):
        if rank < 11:
            i=i+1
            ap10.append(i/rank)
            if rank in eval_avgrank_r:
                ap10_r.append(i/rank)
            if rank in eval_avgrank_m:
                ap10_m.append(i/rank)
            if rank in eval_avgrank_i:
                ap10_i.append(i/rank)
    if ap10:
        apscore10 = sum(ap10)/float(len(ap10))
    else:
        apscore10 = 0
    if ap10_r:
        apscore10_r = sum(ap10_r)/float(len(ap10_r))
    else:
        apscore10_r = 0
    if ap10_m:
        apscore10_m = sum(ap10_m)/float(len(ap10_m))
    else:
        apscore10_m = 0
    if ap10_i:
        apscore10_i = sum(ap10_i)/float(len(ap10_i))
    else:
        apscore10_i = 0

    #Compute overall precision
    pc10 = top10/float(10)
    pc100 = top100/float(100)
    pc500 = top500/float(500)
    pcr = top_r/float(total_match) # R-Precision

    #Compute overall recall
    rc10 = top10/float(total_match)
    rc100 = top100/float(total_match)
    rc500 = top500/float(total_match)

    #Compute specific precision
    pcr_r = top_r_r/float(total_match_r)
    pcr_m = top_r_m/float(total_match_m)
    pcr_i = top_r_i/float(total_match_i)

    #Compute F-scores
    if pc10+rc10:
        f10 = 2*(pc10*rc10)/(pc10+rc10)
    else:
        f10 = 0
    if pc100+rc100:
        f100 = 2*(pc100*rc100)/(pc100+rc100)
    else:
        f100 = 0
    if pc500+rc500:
        f500 = 2*(pc500*rc500)/(pc500+rc500)
    else:
        f500 = 0

    logger.info('Target sentences were the top1 result {} times, were among the top 10 results {} times, among the top 100 results {} times, among the top 500 results {} times, out of {} target documents and {} documents'.format(top1, top10, top100, top500, total_match, total))
    logger.info('Precision @ 10: {}, Precision @ 100: {}, Precision @ 500: {}, R-Precision: {}'.format(pc10, pc100, pc500, pcr))
    logger.info('Recall @ 10: {}, Recall @ 100: {}, Recall @ 500: {}'.format(rc10, rc100, rc500))
    logger.info('F-score @ 10: {}, F-score @ 100: {}, F-score @ 500: {}'.format(f10, f100, f500))
    logger.info('Average Precision @ 10: {}, Average Precision @ 100: {}, Average Precision @ 500: {}'.format(apscore10, apscore100, apscore))

    logger.info('In the top 10 are {} relevant, {} mildly relevant, {} irrelevant and {} unannotated results'.format(top10_r, top10_m, top10_i, 10-(top10_r + top10_m + top10_i)))
    unannotated_top100 = 100-(top100_r + top100_m + top100_i)
    unannotated_top500 = 500-(top500_r + top500_m + top500_i)
    unannotated_top_r = total_match-(top_r_r + top_r_m + top_r_i)
    logger.info('In the top 100 are {} relevant, {} mildly relevant, {} irrelevant and {} unannotated results'.format(top100_r, top100_m, top100_i, unannotated_top100))
    logger.info('In the top 500 are {} relevant, {} mildly relevant, {} irrelevant and {} unannotated results'.format(top500_r, top500_m, top500_i, unannotated_top500))
    logger.info('In the top R are {} relevant, {} mildly relevant, {} irrelevant and {} unannotated results, and R (for relevant and mildly relevant results) is {}'.format(top_r_r, top_r_m, top_r_i, unannotated_top_r, total_match))
    logger.info('R-Precision is {} for relevant + mildly relevant results, {} for relevant results only, {} for mildly relevant results only and {} for irrelevant results only'.format(pcr, pcr_r, pcr_m, pcr_i))

    if eval_avgrank:
        if(len(eval_avgrank) == 1):
            logger.info('Average rank of target sentences: {}. Average distance to query: {}. (range {}--{})'.format(sum(eval_avgrank)/float(len(eval_avgrank)), sum(eval_avg_distance_between)/float(len(eval_avg_distance_between)), min(eval_avg_distance_between), max(eval_avg_distance_between)))
        #else:
            #logger.info('Average rank of target sentences: {} (stdev {}). Average distance to query: {}. (Stdev {}, range {}--{})'.format(sum(eval_avgrank)/float(len(eval_avgrank)), statistics.stdev(eval_avgrank), sum(eval_avg_distance_between)/float(len(eval_avg_distance_between)), statistics.stdev(eval_avg_distance_between), min(eval_avg_distance_between), max(eval_avg_distance_between)))
    else:
        logger.info('No results')


#top 10 matches
    logger.info('Top 100 results for this query:')
    rank = 1
    for match in sims100:
        logger.info('{} (similarity {}) - {} - {}'.format(rank, match[1], match[0], ' '.join(annot_corpus[match[0]])))
        rank=rank+1
#top10 targets
    logger.info('The 10 highest ranked target results for this query:')
    for key in sorted(matches, key=lambda x: matches[x][0]):
        if matches[key][0] < 1001:
            logger.info('{} (rank {}) - {} - {}'.format(key, matches[key][0], matches[key][1], annot_corpus[key]))

#top10 targets
    logger.info('The 10 highest results annotated as irrelevant for this query:')
    for key in sorted(matches_i, key=lambda x: matches_i[x][0]):
        if matches_i[key][0] < 1001:
            logger.info('{} (rank {}) - {} - {}'.format(key, matches_i[key][0], matches_i[key][1], annot_corpus[key]))

    if eval_avgrank:
        #print(eval_avgrank)
        return apscore10, apscore100, apscore, pc10, pc100, pc500, f10, f100, f500, pcr, pcr_r, pcr_m, pcr_i, sum(eval_avgrank_all)/float(len(eval_avgrank_all)), sum(eval_avgrank)/float(len(eval_avgrank)), sum(eval_avg_distance_between_all)/float(len(eval_avg_distance_between_all)), sum(eval_avg_distance_between)/float(len(eval_avg_distance_between))
    else:
        return apscore10, apscore100, apscore, pc10, pc100, pc500, f10, f100, f500, pcr, pcr_r, pcr_m, pcr_i, sum(eval_avgrank_all)/float(len(eval_avgrank_all)), 0, sum(eval_avg_distance_between_all)/float(len(eval_avg_distance_between_all)), 0







In [12]:
#tests2vqueryrun('quinev05_input_word.txt', cwd + '/Data/quinev05_annotU_input_word.txt', 'model.bin', 'telepathy', 1, None)

# Evaluate models on query

In [14]:
path = '/media/joy/Seagate_usb/Models/Sent2Vec/best-word/'
for modelfilename in os.listdir(path):
    print(modelfilename)
    #print(path + modelfilename)
    tests2vqueryrun('quinev05_input_word.txt', cwd + '/Data/quinev05_annotZ_rq1_input_word.txt', path + modelfilename, None, 1, 9097)   


s2vquine05_tune_d128_e100_lr0.25_min0_loss_ns_drop_0.bin
/media/joy/Seagate_usb/Models/Sent2Vec/best-word/s2vquine05_tune_d128_e100_lr0.25_min0_loss_ns_drop_0.bin
16 15 0 0
0.12307692307692308
s2vquine05_tune_d128_e200_lr0.2_min5_loss_ns_drop_0.bin
/media/joy/Seagate_usb/Models/Sent2Vec/best-word/s2vquine05_tune_d128_e200_lr0.2_min5_loss_ns_drop_0.bin
13 10 0 0
0.1
s2vquine05_tune_d256_e100_lr0.25_min0_loss_ns.bin
/media/joy/Seagate_usb/Models/Sent2Vec/best-word/s2vquine05_tune_d256_e100_lr0.25_min0_loss_ns.bin
15 13 0 0
0.11538461538461539
s2vquine05_tune_d256_e100_lr0.25_min5_loss_ns.bin
/media/joy/Seagate_usb/Models/Sent2Vec/best-word/s2vquine05_tune_d256_e100_lr0.25_min5_loss_ns.bin
19 14 0 0
0.14615384615384616
s2vquine05_tune_d256_e100_lr0.3_min0_loss_ns.bin
/media/joy/Seagate_usb/Models/Sent2Vec/best-word/s2vquine05_tune_d256_e100_lr0.3_min0_loss_ns.bin
16 14 0 0
0.12307692307692308
s2vquine05_tune_d256_e200_lr0.2_min5_loss_ns_drop_0.bin
/media/joy/Seagate_usb/Models/Sent2Vec/be

In [ ]:
def parselogs(indir, outfile):
    with codecs.open(outfile, "w", encoding="utf-8") as o:
        csvhead = ["filename","p10","p100","p500","f10","f100","f500","map10","map100","map500","mpr","rank","rank500","dist","dist500"]
        o.write('id,' + ','.join(csvhead) + '\n')
        run = 0
        for filename in os.listdir(indir): 
            #if not re.match("d2vquine05test.word*.txt", filename):
                #continue
            if os.stat(indir + '/' + filename).st_size == 0: #not empty files
                continue
            evaldict = {}
            evaldict['filename'] = filename
            res = 0
            res = re.search('\.v(\d+\.?\d*)', filename)
#             if res:
#                 evaldict['vectorsize'] = res.group(1)
#             res = re.search('\.e(\d+\.?\d*)', filename)
#             if res:
#                 evaldict['epochs'] = res.group(1)
#             res = re.search('\.w(\d+\.?\d*)', filename)
#             if res:
#                 evaldict['window'] = res.group(1)
#             res = re.search('\.s(\d+\.?\d*)', filename)
#             if res:
#                 evaldict['subsample'] = res.group(1)
#             res = re.search('\.c(\d+\.?\d*)', filename)
#             if res:
#                 evaldict['cutoff'] = res.group(1)
#             res = re.search('\.a(\d+\.?\d*)', filename)
#             if res:
#                 evaldict['alpha'] = res.group(1)
#             res = re.search('\.t(\d+\.?\d*)', filename)
#             if res:
#                 evaldict['bithreshold'] = res.group(1)
#             res = re.search('\.mc(\d+\.?\d*)', filename)
#             if res:
#                 evaldict['bicutoff'] = res.group(1)
#             res = re.search('\.d([a-z]+)', filename)
#             if res:
#                 evaldict['method'] = res.group(1)
#             res = re.search('\.bi', filename)
#             if res:
#                 evaldict['bigram'] = res.group(1)
#             res = re.match('\.hs', filename)
#             if res:
#                 evaldict['hs'] = 1
#             else:
#                 evaldict['hs'] = 0
            res = re.match('_both', filename)
#             if res:
#                 evaldict['eval'] = 'both'
#             else:
#                 evaldict['eval'] = ''

            with codecs.open(indir + '/' + filename, "r", encoding="utf-8") as f:
                print(indir + '/' + filename)
                #data = mmap.mmap(f.fileno(), 0, prot=mmap.PROT_READ)
                data = f.readlines()
                data = ' '.join(data)
                print(len(data))
                res = re.search('Mean Precision @ 10: (\d+\.?\d*), Mean Precision @ 100: (\d+\.?\d*), Mean Precision @ 500: (\d+\.?\d*)', data)
                if res:
                    evaldict['p10'] = res.group(1)
                    evaldict['p100'] = res.group(2)
                    evaldict['p500'] = res.group(3)
                
                res = re.search('Mean F-score @ 10: (\d+\.?\d*), Mean F-score @ 100: (\d+\.?\d*), Mean F-score @ 500: (\d+\.?\d*)', data)
                if res:
                    evaldict['f10'] = res.group(1)
                    evaldict['f100'] = res.group(2)
                    evaldict['f500'] = res.group(3)

                res = re.search('Mean Average Precision @ 10: (\d+\.?\d*), Mean Average Precision @ 100: (\d+\.?\d*), Mean Average Precision @ 500: (\d+\.?\d*)', data)
                if res:
                    evaldict['map10'] = res.group(1)
                    evaldict['map100'] = res.group(2)
                    evaldict['map500'] = res.group(3)

                res = re.search('Mean Precision @ R: (\d+\.?\d*)', data)
                if res:
                    evaldict['mpr'] = res.group(1)

                res = re.search('Average rank of targets: (\d+\.?\d*)', data)
                if res:
                    evaldict['rank'] = res.group(1)
                    
                res = re.search('Average rank of top 500 targets: (\d+\.?\d*)', data)
                if res:
                    evaldict['rank500'] = res.group(1)

                res = re.search('Average distance of target to query: (\d+\.?\d*)', data)
                if res:
                    evaldict['dist'] = res.group(1)

                res = re.search('Average distance of top 500 targets to query: (\d+\.?\d*)', data)
                if res:
                    evaldict['dist500'] = res.group(1)

#                 #additive vector
#                 res = re.search('Testing model with additive query vector[\s\S]*Mean Precision @ 10: (\d+\.?\d*), Mean Precision @ 100: (\d+\.?\d*), Mean Precision @ 500: (\d+\.?\d*)', data)
#                 if res:
#                     evaldict['p10add'] = res.group(1)
#                     evaldict['p100add'] = res.group(2)
#                     evaldict['p500add'] = res.group(3)
                
#                 res = re.search('Testing model with additive query vector[\s\S]*Mean F-score @ 10: (\d+\.?\d*), Mean F-score @ 100: (\d+\.?\d*), Mean F-score @ 500: (\d+\.?\d*)', data)
#                 if res:
#                     evaldict['f10add'] = res.group(1)
#                     evaldict['f100add'] = res.group(2)
#                     evaldict['f500add'] = res.group(3)

#                 res = re.search('Testing model with additive query vector[\s\S]*Mean Average Precision @ 10: (\d+\.?\d*), Mean Average Precision @ 100: (\d+\.?\d*), Mean Average Precision @ 500: (\d+\.?\d*)', data)
#                 if res:
#                     evaldict['map10add'] = res.group(1)
#                     evaldict['map100add'] = res.group(2)
#                     evaldict['map500add'] = res.group(3)

#                 res = re.search('Testing model with additive query vector[\s\S]*Mean Precision @ R: (\d+\.?\d*)', data)
#                 if res:
#                     evaldict['mpradd'] = res.group(1)

#                 res = re.search('Testing model with additive query vector[\s\S]*Average rank of targets: (\d+\.?\d*)', data)
#                 if res:
#                     evaldict['rankadd'] = res.group(1)
                    
#                 res = re.search('Testing model with additive query vector[\s\S]*Average rank of top 500 targets: (\d+\.?\d*)', data)
#                 if res:
#                     evaldict['rank500add'] = res.group(1)

#                 res = re.search('Testing model with additive query vector[\s\S]*Average distance of target to query: (\d+\.?\d*)', data)
#                 if res:
#                     evaldict['distadd'] = res.group(1)

#                 res = re.search('Testing model with additive query vector[\s\S]*Average distance of top 500 targets to query: (\d+\.?\d*)', data)
#                 if res:
#                     evaldict['dist500add'] = res.group(1)




                #res = re.search('Part 1 term was the 1st nearest neighbour (\d+\.?\d*) times', data)
                #if res:
                #    evaldict['top'] = res.group(1)
                #res = re.search('and was among the top 10 nearest neighbours (\d+\.?\d*) times', data)
                #if res:
                #    evaldict['top10'] = res.group(1)
                #res = re.search('Part 1 baseline term was the 1st nearest neighbour of part 2 baseline (\d+\.?\d*) times', data)
                #if res:
                #    evaldict['topbase'] = res.group(1)
                #res = re.search('and was among the top 10 nearest neighbours of baseline (\d+\.?\d*) times', data)
                #if res:
                #    evaldict['top10base'] = res.group(1)
                #res = re.search(', out of (\d+\.?\d*) terms', data)
                #if res:
                #    evaldict['total'] = res.group(1)
            run = run + 1

            #output file (csv)
            o.write(str(run))
            for column in csvhead:
                if column in evaldict:
                    o.write(','+ str(evaldict[column]))
                else:
                    o.write(',')
            o.write('\n')

In [ ]:
#parselogs('/media/joy/Seagate_usb/Logs/ns/Test/word/system_of_the_world', '/media/joy/Seagate_usb/Logs/ns/Test/word/system_of_the_world.xls')